In [ ]:
# %pip install transformers torch

In [31]:
text = """A sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements.
She was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the
right and upwards due to the sustained contraction of the neck muscles. There was a sideways bending of the back in the lumbar region. To counter the abnormal positioning
of the back and neck, she would keep her limbs in a specific position to allow her body weight to be supported. Due to the restrictions with the body movements at the neck
and in the lumbar region, she would require assistance in standing and walking. She would require her parents to help her with daily chores, including all activities of self-care.
She had been experiencing these difficulties for the past four months since when she was introduced to olanzapine tablets for the control of her exacerbated mental illness.
This was not her first experience with this drug over the past seven years since she had been diagnosed with bipolar affective disorder.
Her first episode of the affective disorder was that of mania at the age of eleven which was managed with the use of olanzapine tablets in 2.5–10 mg doses per day at different times.
The patient developed pain and discomfort in her neck within the second week of being put on tablet olanzapine at a dose of 5 mg per day.
This was associated with a sustained and abnormal contraction of the neck muscles that would pull her head to the right in an upward direction.
These features had persisted for the first three years of her illness with a varying intensity, distress, and dysfunction which would tend to correlate with the dose of olanzapine.
Apart from a brief period of around three weeks when she was given tablet trihexyphenidyl 4 mg per day for rigidity in her upper limbs, she was not prescribed any other psychotropic medication.
The rigidity showed good response to this medication which was subsequently"""

**CLINICAL-BERT NER + SPACY**

In [32]:
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "samrawal/bert-base-uncased_clinical-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

ner_results = ner_pipeline(text)

# Print results
for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}) → score: {entity['score']:.2f}")

# Map of masked intervals
to_mask = []
for entity in ner_results:
    if entity["entity_group"] in {"problem", "treatment"}:
        to_mask.append((entity["start"], entity["end"]))

# Function to mask words mantaining first two letter (eg. nack --> na**)
def mask_word(word):
    if len(word) <= 2:
        return word
    return word[:2] + "*" * (len(word) - 2)

# Mask words in specif ranges
masked_text = ""
i = 0
for start, end in sorted(to_mask):
    masked_text += text[i:start]
    original = text[start:end]
    masked = re.sub(r'\b\w+\b', lambda m: mask_word(m.group()), original)
    masked_text += masked
    i = end

masked_text += text[i:]

print("\nMasked text (first two letter are visible for PROBLEM/TREATMENT):\n")
print(masked_text)

Device set to use cpu


discomfort in the neck and lower back (problem) → score: 1.00
restriction of body movements (problem) → score: 0.71
the sustained contraction of the neck muscles (problem) → score: 1.00
a sideways bending of the back (problem) → score: 0.96
the abnormal positioning of the back and neck (problem) → score: 0.98
the restrictions (problem) → score: 0.55
the body (problem) → score: 0.60
these difficulties (problem) → score: 0.99
olanza (treatment) → score: 0.84
##pine tablets (treatment) → score: 0.76
her exacerbated mental illness (problem) → score: 0.99
this drug (treatment) → score: 0.95
bipolar affective disorder (problem) → score: 1.00
the affective disorder (problem) → score: 1.00
mania (problem) → score: 1.00
olanzapine tablets (treatment) → score: 1.00
pain and (problem) → score: 0.94
discomfort in her neck (problem) → score: 0.88
tablet olanzapine (treatment) → score: 0.99
a sustained and abnormal contraction of the neck muscles (problem) → score: 1.00
these features (problem) → sc

In [ ]:
import spacy
spacy.cli.download("en_core_web_lg")

In [40]:
nlp = spacy.load("en_core_web_lg")

text = masked_text

doc = nlp(text)

print("Entity found:")
for ent in doc.ents:
    print(f"Text: '{ent.text}' - Type: {ent.label_} - Start: {ent.start_char}, End: {ent.end_char}")

# Entity type to mask
entities_to_mask = {"PERSON", "NORP", "DATE", "CARDINAL", "QUANTITY"}

# List of masked intervals
to_mask = []
other_to_mask = []
for ent in doc.ents:
    if ent.label_ in entities_to_mask:
        to_mask.append((ent.start_char, ent.end_char))
    if ent.label_ in ["CARDINAL", "QUANTITY"]:
        other_to_mask.append((ent.start_char, ent.end_char))

# Mask words in specif ranges
masked_text_new = ""
i = 0
for start, end in sorted(to_mask):
    if (start, end) in other_to_mask:
        masked_text_new += text[i:start]
        masked_text_new += '*'
        i = end
        continue
    masked_text_new += text[i:start]
    original = text[start:end]
    masked = re.sub(r'\b\w+\b', lambda m: mask_word(m.group()), original)
    masked_text_new += masked
    i = end

masked_text_new += text[i:]

print("\nMasked text (PERSON, NORP, DATE, CARDINAL, QUANTITY):\n")
print(masked_text_new)

Entità trovate:
Testo: 'sixteen year-old' - Tipo: DATE - Inizio: 2, Fine: 18
Testo: 'Outpatient' - Tipo: DATE - Inizio: 42, Fine: 52
Testo: 'the past four months' - Tipo: DATE - Inizio: 904, Fine: 924
Testo: 'first' - Tipo: ORDINAL - Inizio: 1045, Fine: 1050
Testo: 'the past seven years' - Tipo: DATE - Inizio: 1082, Fine: 1102
Testo: 'first' - Tipo: ORDINAL - Inizio: 1169, Fine: 1174
Testo: 'the age of eleven' - Tipo: DATE - Inizio: 1230, Fine: 1247
Testo: '2.5–10' - Tipo: CARDINAL - Inizio: 1304, Fine: 1310
Testo: 'the second week' - Tipo: DATE - Inizio: 1409, Fine: 1424
Testo: '5 mg' - Tipo: QUANTITY - Inizio: 1472, Fine: 1476
Testo: 'the first three years' - Tipo: DATE - Inizio: 1663, Fine: 1684
Testo: 'around three weeks' - Tipo: DATE - Inizio: 1840, Fine: 1858
Testo: '4 mg' - Tipo: QUANTITY - Inizio: 1901, Fine: 1905

Testo mascherato (PERSON, NORP, DATE, CARDINAL, QUANTITY):

A si***** ye**-ol* girl, presented to our Ou******** department with the complaints of di******** in th* 

**GPT 4o**

**PROMPT**

Sei un assistente AI specializzato nell'anonimizzare le informazioni sensibili in ambito medico secondo gli standard imposti dal GDPR rispetto alle leggi 4 e 9.

Dato il seguente testo:

"A sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements. She was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the right and upwards due to the sustained contraction of the neck muscles. There was a sideways bending of the back in the lumbar region. To counter the abnormal positioning of the back and neck, she would keep her limbs in a specific position to allow her body weight to be supported. Due to the restrictions with the body movements at the neck and in the lumbar region, she would require assistance in standing and walking. She would require her parents to help her with daily chores, including all activities of self-care.
She had been experiencing these difficulties for the past four months since when she was introduced to olanzapine tablets for the control of her exacerbated mental illness. This was not her first experience with this drug over the past seven years since she had been diagnosed with bipolar affective disorder. Her first episode of the affective disorder was that of mania at the age of eleven which was managed with the use of olanzapine tablets in 2.5–10 mg doses per day at different times. The patient developed pain and discomfort in her neck within the second week of being put on tablet olanzapine at a dose of 5 mg per day. This was associated with a sustained and abnormal contraction of the neck muscles that would pull her head to the right in an upward direction. These features had persisted for the first three years of her illness with a varying intensity, distress, and dysfunction which would tend to correlate with the dose of olanzapine. Apart from a brief period of around three weeks when she was given tablet trihexyphenidyl 4 mg per day for rigidity in her upper limbs, she was not prescribed any other psychotropic medication. The rigidity showed good response to this medication which was subsequently"

Applica la tecnica del Data Masking (mascheramento parziale delle entità sensibili). Le entità sensibili riguardano:
- Età, nome e sesso del paziente
- Diagnosi psichiatrica e storia clinica dettagliata
- Terapie farmacologiche specifiche (olanzapina, trihexyphenidyl) e relative quantità
- Effetti collaterali e sintomi neurologici/motori
- Dati funzionali e relativi all’autonomia personale
- Cronologia clinica e terapeutica dettagliata

Ad esempio data la frase:

"A sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements. Her first episode of the affective disorder was that of mania at the age of eleven which was managed with the use of olanzapine tablets in 2.5–10 mg doses per day at different times."

Diventa:

```A si***** ye**-ol* girl, presented to our Ou******** department with the complaints of di******** in th* ne** an* lo*** ba** as well as re********* of bo** mo*******.Her first episode of th* af******* di****** was that of ma*** at th* ag* of el**** which was managed with the use of ol******** ta***** in * mg doses per day at different times.```

**RESULT**

```A si****** ye**-ol* gi**, presented to our Ou******** department with the complaints of di******** in th* ne** an* lo*** ba** as well as re********* of bo** mo*******. She was not able to maintain an er*** po****** and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the ri*** and up***** due to the su******* co******** of the ne** mu******. There was a si****** be****** of the ba** in the lu****re*****. To counter the ab****** po********* of the ba** and ne**, she would keep her li*** in a sp****** po****** to allow her bo** we**** to be supported. Due to the re********* with the bo** mo******* at the ne** and in the lu**** re*****, she would require assistance in standing and walking. She would require her pa****** to help her with da** ch****, including all ac********* of se**-ca**. She had been experiencing these difficulties for the past fo** mo**** since when she was introduced to ol******** ta***** for the control of her ex********* me**** ill****. This was not her first experience with this dr** over the past se*** ye*** since she had been diagnosed with bi***** af******* di******. Her first episode of th* af******* di****** was that of ma*** at th* ag* of el**** which was managed with the use of ol******** ta***** in –* mg doses per day at different times. The patient developed pa** and di******** in her ne** within the se**** we** of being put on tablet ol******** at a dose of * mg per day. This was associated with a su******* and ab****** co******** of the ne** mu****** that would pull her head to the ri*** in an up***** di*******. These features had persisted for the first th*** ye*** of her ill**** with a varying intensity, distress, and dysfunction which would tend to correlate with the dose of ol********. Apart from a brief period of around th*** we*** when she was given tablet tr************* * mg per day for ri****** in her up*** li***, she was not prescribed any other ps********* me********. The ri****** showed good response to this me******** which was subsequently.```